In [83]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt 
%matplotlib inline

In [84]:
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test
from lifelines import KaplanMeierFitter

In [85]:
#loading data
dataset_survival = pd.read_csv('dataset_survival.csv', encoding="latin1", low_memory=False)
dataset_survival = dataset_survival.iloc[: , 1:]

In [86]:
dataset_survival.head()

,DON_RACE,DON_A1,DON_A2,DON_B1,DON_B2,DON_DR1,DON_DR2,PERS_ID,CAN_RACE,REC_A1,...,PERS_OPTN_DEATH_DT,PERS_RESTRICT_DEATH_DT,TFL_DEATH_DT,TFL_LAFUDATE,TFL_ENDTXFU,PS,KGL,DCGL,GL,DAYS
0,White,1.0,11.0,35.0,60.0,1.0,8.0,3759020.0,White,11.0,...,NaN,NaN,NaN,2018-04-12,2018-11-21,0.0,0.0,0,0.0,1969
1,White,24.0,68.0,60.0,62.0,4.0,13.0,3147238.0,Black or African American,31.0,...,NaN,NaN,NaN,2017-10-24,2018-10-20,0.0,1.0,0,1.0,1831
2,White,1.0,2.0,7.0,8.0,17.0,4.0,2599447.0,Black or African American,203.0,...,NaN,NaN,NaN,2017-02-09,2018-10-01,0.0,0.0,0,0.0,1593
3,Black or African American,30.0,74.0,8.0,63.0,17.0,7.0,3257192.0,Black or African American,1.0,...,NaN,NaN,NaN,2013-01-08,2018-12-31,0.0,1.0,0,1.0,9
4,White,2.0,11.0,51.0,55.0,4.0,11.0,2619730.0,White,3.0,...,2017-09-05,NaN,NaN,2015-01-19,2018-10-06,1.0,1.0,0,1.0,836


In [87]:
dataset_survival.columns

Index(['DON_RACE', 'DON_A1', 'DON_A2', 'DON_B1', 'DON_B2', 'DON_DR1',
       'DON_DR2', 'PERS_ID', 'CAN_RACE', 'REC_A1', 'REC_A2', 'REC_B1',
       'REC_B2', 'REC_DR1', 'REC_DR2', 'DON_A1_HR', 'DON_A2_HR', 'DON_B1_HR',
       'DON_B2_HR', 'DON_DR1_HR', 'DON_DR2_HR', 'REC_A1_HR', 'REC_A2_HR',
       'REC_B1_HR', 'REC_B2_HR', 'REC_DR1_HR', 'REC_DR2_HR', 'AB_HMS',
       'AB_EMS', 'AB_AMS', 'DR_HMS', 'DR_EMS', 'DR_AMS', 'DR_alphaHMS',
       'DR_alphaEMS', 'DR_alphaAMS', 'DR_betaHMS', 'DR_betaEMS', 'DR_betaAMS',
       'Avg_EMS', 'Avg_HMS', 'Avg_AMS', 'DON_EPLETS', 'REC_EPLETS',
       'AbvMismatch', 'OthMismatch', 'Abv_Desc', 'Oth_Desc', 'TFL_GRAFT_DT',
       'REC_TX_DT', 'TFL_LASTATUS', 'CAN_GENDER', 'REC_AGE_AT_TX',
       'REC_FAIL_DT', 'PERS_RELIST', 'PERS_RETX', 'REC_RESUM_MAINT_DIAL_DT',
       'PERS_SSA_DEATH_DT', 'PERS_OPTN_DEATH_DT', 'PERS_RESTRICT_DEATH_DT',
       'TFL_DEATH_DT', 'TFL_LAFUDATE', 'TFL_ENDTXFU', 'PS', 'KGL', 'DCGL',
       'GL', 'DAYS'],
      dtype='object')

In [88]:
#SOME OF THE NEW FEATURES BEING CHECKED FOR SIGNIFICANCE TO OUR COX PH MODEL
REC_PROD_URINE_GT40_24HRS	Kidney Produced > 40ml of Urine in First 24 Hours

DON_HIST_DIAB  History of Diabetes
DON_HIST_HYPERTEN History of Hypertension
DON_HIST_CANCER  History of Cancer
DON_HIGH_CREAT  Donor serum creatinine > 1.5 (1= yes, 0= no)
DON_HTN  Donor history of hypertension (1= yes, 0= no)

DON_HIST_CIGARETTE_GT20_PKYR
DON_HIST_COCAINE
DON_HIST_OTHER_DRUG

DON_RELATIONSHIP_TY Living Donor/s Relationship to Recipient
DON_TY  Donor Type (C-deceased, L-living)

REC_HOSP_90_DAYS Was patient hospitalizied during the last 90 days prior to the transplant admission

REC_MED_COND Medical Condition

REC_MM_EQUIV_CUR Num of hla mismatches - current equivalent match
REC_MM_EQUIV_TX  Num of hla mismatches - current match @ tx

REC_ORG_RECEIVED_ON  Kidney(s) received on ice, pump

SyntaxError: invalid syntax (<ipython-input-88-fd7343a1968b>, line 2)

In [89]:
#transplant info dataset
tx_ki = pd.read_sas('/home/kidneyData/DATA_SP_2020/tx_ki.sas7bdat')

In [90]:
tx_ki.head()

,ORG_TY,PERS_ID,REC_TX_DT,REC_POSTX_LOS,PX_ID,REC_A_MM_EQUIV_TX,REC_A_MM_EQUIV_CUR,REC_B_MM_EQUIV_TX,REC_B_MM_EQUIV_CUR,REC_DR_MM_EQUIV_TX,...,TFL_COD,PERS_RELIST,PERS_RETX,PERS_RETX_TRR_ID,TFL_LAFUDATE,REC_MM_EQUIV_TX,REC_MM_EQUIV_CUR,PERS_OPTN_COD,DONOR_ID,DONOR_ID_VESSEL
0,b'KI',4956632.0,2012-11-20,5.0,853031.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaT,NaT,NaN,2018-04-12,3.0,3.0,NaN,419584.0,NaN
1,b'KI',4787788.0,2012-10-19,5.0,736755.0,0.0,0.0,0.0,0.0,0.0,...,NaN,2013-08-21,NaT,NaN,2017-10-24,0.0,0.0,NaN,418721.0,NaN
2,b'KI',4059261.0,2012-09-30,6.0,575654.0,2.0,2.0,2.0,2.0,0.0,...,NaN,NaT,NaT,NaN,2017-02-09,4.0,4.0,NaN,417693.0,NaN
3,b'KI',4893631.0,2012-12-30,9.0,785678.0,1.0,1.0,2.0,2.0,2.0,...,NaN,2013-01-02,2018-02-07,773433.0,2013-01-08,5.0,5.0,NaN,422447.0,NaN
4,b'KI',4573989.0,2012-10-05,2.0,664555.0,1.0,1.0,2.0,2.0,1.0,...,NaN,NaT,NaT,NaN,2015-01-19,4.0,4.0,NaN,417628.0,NaN


In [91]:
#merge with tx_ki dataset
dataset_survival = dataset_survival.merge(tx_ki, left_index=True, right_index=True,
             on='PERS_ID', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [92]:
dataset_reg = dataset_survival[[
 'REC_AGE_AT_TX', 
 'DON_AGE',  
 'CAN_LAST_SRTR_PEAK_PRA',
 'REC_DIAL_TY',
 'DON_RACE', 
 'CAN_RACE', 
 'CAN_GENDER',
 'DON_GENDER', 
 'GL',  
 'DAYS', 
 'REC_BMI',
 'REC_HGT_CM',
 'CAN_HGT_CM',
 'CAN_WGT_KG',
 'REC_WGT_KG',
 'DON_HGT_CM',
 'DON_WGT_KG',
 'REC_PRIMARY_PAY', 
 'REC_COLD_ISCH_TM',
 'REC_TX_DT',
 'REC_PROD_URINE_GT40_24HRS',
 'DON_HIGH_CREAT', 
 'DON_HIST_CIGARETTE_GT20_PKYR',
 'DON_HIST_COCAINE',
 'DON_HIST_OTHER_DRUG', 
 'DON_TY',
 'REC_MALIG',
 'REC_HOSP_90_DAYS',
 'Avg_HMS',
 'AbvMismatch',
 'REC_MM_EQUIV_CUR',
 'REC_MM_EQUIV_TX',
 'DON_HTN',
 'DON_HIST_CANCER',
 'DON_RELATIONSHIP_TY',
 'REC_MED_COND',
 'REC_ORG_RECEIVED_ON'
 ]]

#### DON_BMI calculation

In [93]:
pd.options.mode.chained_assignment = None

In [94]:
#fill in missing value
dataset_reg['DON_WGT_KG'] = dataset_reg['DON_WGT_KG'].fillna(dataset_reg['DON_WGT_KG'].mean())

In [95]:
#fill in missing value
dataset_reg['DON_HGT_CM'] = dataset_reg['DON_HGT_CM'].fillna(dataset_reg['DON_HGT_CM'].mean())

In [96]:
def bmi(x):
    return x.DON_WGT_KG/((x.DON_HGT_CM/100)**2)

dataset_reg['DON_BMI'] = dataset_reg.apply(lambda x: bmi(x), axis=1)

In [97]:
dataset_reg = dataset_reg[[
 'REC_AGE_AT_TX', 
 'DON_AGE',  
 'CAN_LAST_SRTR_PEAK_PRA',
 'DON_RACE', 
 'CAN_RACE', 
 'CAN_GENDER',
 'DON_GENDER', 
 'GL',  
 'DAYS', 
 'DON_BMI',
 'REC_BMI',
 'REC_PRIMARY_PAY',  
 'REC_COLD_ISCH_TM',
 'REC_TX_DT',
 'REC_PROD_URINE_GT40_24HRS',
 'DON_HIGH_CREAT', 
 'DON_HIST_CIGARETTE_GT20_PKYR',
 'DON_HIST_COCAINE',
 'DON_HIST_OTHER_DRUG', 
 'DON_TY',
 'REC_MALIG',
 'REC_HOSP_90_DAYS',
 'Avg_HMS',
 'AbvMismatch',
 'REC_MM_EQUIV_CUR',
 'REC_MM_EQUIV_TX',
 'DON_HTN',
 'DON_HIST_CANCER',
 'DON_RELATIONSHIP_TY',
 'REC_MED_COND',
 'REC_ORG_RECEIVED_ON']]

In [98]:
dataset_reg.head()

,REC_AGE_AT_TX,DON_AGE,CAN_LAST_SRTR_PEAK_PRA,DON_RACE,CAN_RACE,CAN_GENDER,DON_GENDER,GL,DAYS,DON_BMI,...,REC_HOSP_90_DAYS,Avg_HMS,AbvMismatch,REC_MM_EQUIV_CUR,REC_MM_EQUIV_TX,DON_HTN,DON_HIST_CANCER,DON_RELATIONSHIP_TY,REC_MED_COND,REC_ORG_RECEIVED_ON
0,30.0,28.0,6.0,White,White,b'M',b'F',0.0,1969,30.725868,...,b'N',6.0500,19,3.0,3.0,NaN,1.0,4.0,3.0,1.0
1,26.0,53.0,13.0,White,Black or African American,b'M',b'F',1.0,1831,23.336063,...,b'N',17.7000,21,0.0,0.0,0.0,1.0,NaN,3.0,1.0
2,42.0,53.0,0.0,White,Black or African American,b'M',b'F',0.0,1593,27.365657,...,b'N',10.1750,16,4.0,4.0,0.0,1.0,NaN,3.0,1.0
3,62.0,17.0,NaN,Black or African American,Black or African American,b'F',b'F',1.0,9,27.412446,...,b'N',10.0875,17,5.0,5.0,0.0,1.0,NaN,3.0,1.0
4,68.0,59.0,19.0,White,White,b'M',b'M',1.0,836,40.740741,...,b'N',8.8250,11,4.0,4.0,1.0,1.0,NaN,3.0,2.0


### Preprocessing some features

##### REC_AGE_AT_TX, DON AGE

In [99]:
# REC_AGE_AT_TX, DON AGE # changed to 18-34, 34-49, 50 or more
def age_group(age):
    if age <= 34:  
        return "18-34"
    elif age <= 49: 
        return "35 - 49"
    else:           
        return "> 45"


dataset_reg['REC_AGE_AT_TX'] = dataset_reg['REC_AGE_AT_TX'].apply(age_group)
dataset_reg['DON_AGE'] = dataset_reg['DON_AGE'].apply(age_group)

##### CAN_LAST_SRTR_PEAK_PRA

In [100]:
#fill in missing value
dataset_reg['CAN_LAST_SRTR_PEAK_PRA'] = dataset_reg['CAN_LAST_SRTR_PEAK_PRA'].fillna(dataset_reg['CAN_LAST_SRTR_PEAK_PRA'].mean())

In [101]:
def PRA_group(age):
    if age <= 19:  
        return "0-19"
    elif age <= 95: 
        return "19 - 95"
    else:           
        return "> 95"


dataset_reg['CAN_LAST_SRTR_PEAK_PRA'] = dataset_reg['CAN_LAST_SRTR_PEAK_PRA'].apply(PRA_group)

##### RACE

In [102]:
def race_label(race):
    if race == "White":
        return "White"
    elif race == "Black or African American": 
        return "Black"
    else:
        return "All Others"

# Apply race_label function to DON_RACE column
dataset_reg['DON_RACE'] = dataset_reg['DON_RACE'].apply(race_label)
# Apply race_label function to CAN_RACE column
dataset_reg['CAN_RACE'] = dataset_reg['CAN_RACE'].apply(race_label)

##### REC BMI, DON BMI

In [103]:
#fill in missing value
dataset_reg['REC_BMI'] = dataset_reg['REC_BMI'].fillna(dataset_reg['REC_BMI'].mean())

In [104]:
def BMI_group(bmi):
    if bmi <= 30:  
        return "0-30"
    else:           
        return "> 30"


dataset_reg['REC_BMI'] = dataset_reg['REC_BMI'].apply(BMI_group)
dataset_reg['DON_BMI'] = dataset_reg['DON_BMI'].apply(BMI_group)

##### REC_PRIMARY_PAY

In [105]:
def pay_label(pay):
    if pay == 1:
        return "Private insurance"
    elif pay == 2: 
        return "Public insurance"
    else:
        return "All Others"


dataset_reg['REC_PRIMARY_PAY'] = dataset_reg['REC_PRIMARY_PAY'].apply(pay_label)

##### REC_COLD_ISCH_TM

In [106]:
def cold_group(time):
    if time <= 24:  
        return "0-24"
    else:           
        return "> 24"

    
dataset_reg['REC_COLD_ISCH_TM'] = dataset_reg['REC_COLD_ISCH_TM'].apply(cold_group)

##### REC_TX_DT

In [107]:
def transp_era(time):
    if time < "2010-01-01": 
        return "Before 2010"
    else:           
        return "2011 and after"

    
dataset_reg['REC_TX_DT'] = dataset_reg['REC_TX_DT'].apply(transp_era)

In [108]:
dataset_reg.REC_TX_DT.value_counts()

2011 and after    71865
Before 2010        9139
Name: REC_TX_DT, dtype: int64

#### DON_HIGH_CREAT

In [109]:
pd.options.mode.chained_assignment = None

In [110]:
def DON_HIGH_CREAT_label(label):
    if label == 0:
        return "No"
    elif label == 1: 
        return "Yes"
    else:
        return "Unknown"

In [111]:
dataset_reg['DON_HIGH_CREAT'] = dataset_reg['DON_HIGH_CREAT'].apply(DON_HIGH_CREAT_label)

#### DON_HIST_CIGARETTE_GT20_PKYR

In [112]:
def CIGARETTE_label(label):
    if label == b'N':
        return "No"
    elif label == b'Y': 
        return "Yes"
    else:
        return "Unknown"

In [113]:
dataset_reg['DON_HIST_CIGARETTE_GT20_PKYR'] = dataset_reg['DON_HIST_CIGARETTE_GT20_PKYR'].apply(CIGARETTE_label)

#### DON_HIST_COCAINE

In [114]:
def COC_label(label):
    if label == b'N':
        return "No"
    elif label == b'Y': 
        return "Yes"
    else:
        return "Unknown"

In [115]:
dataset_reg['DON_HIST_COCAINE'] = dataset_reg['DON_HIST_COCAINE'].apply(COC_label)

#### DON_HIST_OTHER_DRUG

In [116]:
def OTHER_DRUG_label(label):
    if label == b'N':
        return "No"
    elif label == b'Y': 
        return "Yes"
    else:
        return "Unknown"

In [117]:
dataset_reg['DON_HIST_OTHER_DRUG'] = dataset_reg['DON_HIST_OTHER_DRUG'].apply(OTHER_DRUG_label)

#### REC_MALIG

In [118]:
def REC_MALIG_label(label):
    if label == b'N':
        return "No"
    elif label == b'Y': 
        return "Yes"
    else:
        return "Unknown"

In [119]:
dataset_reg['REC_MALIG'] = dataset_reg['REC_MALIG'].apply(REC_MALIG_label)

#### REC_PROD_URINE_GT40_24HRS

In [120]:
def REC_PROD_URINE_GT40_24HRS_label(label):
    if label == b'N':
        return "No"
    elif label == b'Y': 
        return "Yes"
    else:
        return "Unknown"

In [121]:
dataset_reg['REC_PROD_URINE_GT40_24HRS'] = dataset_reg['REC_PROD_URINE_GT40_24HRS'].apply(REC_PROD_URINE_GT40_24HRS_label)

#### REC_HOSP_90_DAYS

In [122]:
def REC_HOSP_90_DAYS_label(label):
    if label == b'N':
        return "not hosptalized"
    elif label == b'Y': 
        return "hosptalized"
    else:
        return "Unknown"

In [123]:
dataset_reg['REC_HOSP_90_DAYS'] = dataset_reg['REC_HOSP_90_DAYS'].apply(REC_HOSP_90_DAYS_label)

#### AbvMismatch

In [124]:
#dataset_reg.AbvMismatch.value_counts()

In [125]:
#10 used as cut off
def eplet_group(mismatch):
    if mismatch <= 5: 
        return "<= 5"
    elif mismatch <= 10: 
        return "5-10"
    else:
        return ">10"

In [126]:
dataset_reg['AbvMismatch'] = dataset_reg['AbvMismatch'].apply(eplet_group)

In [127]:
#tried <= 10

### Avg_HMS

In [128]:
def Avg_HMS_group(Avg_AMS):
    if Avg_AMS <= 1.8: 
        return "<= 1.8"
    else:
        return ">1.8"

In [129]:
dataset_reg['Avg_HMS'] = dataset_reg['Avg_HMS'].apply(Avg_HMS_group)

#### REC_MM_EQUIV_CUR

In [130]:
dataset_reg.REC_MM_EQUIV_CUR.value_counts()

5.0    22363
4.0    18641
3.0    14087
6.0    11386
2.0     6333
0.0     5415
1.0     1938
Name: REC_MM_EQUIV_CUR, dtype: int64

In [131]:
def REC_MM_EQUIV_CUR_label(mismatch):
    if mismatch <= 2: 
        return "<= 2"
    elif mismatch <= 4: 
        return "2-4"
    else:
        return ">4"

In [132]:
dataset_reg['REC_MM_EQUIV_CUR'] = dataset_reg['REC_MM_EQUIV_CUR'].apply(REC_MM_EQUIV_CUR_label)

In [133]:
dataset_reg.REC_MM_EQUIV_CUR.value_counts()

>4      34590
2-4     32728
<= 2    13686
Name: REC_MM_EQUIV_CUR, dtype: int64

### REC_MM_EQUIV_TX

In [134]:
dataset_reg.REC_MM_EQUIV_TX.value_counts()

5.0    22178
4.0    18833
3.0    14292
6.0    10954
2.0     6465
0.0     5665
1.0     1776
Name: REC_MM_EQUIV_TX, dtype: int64

In [135]:

def REC_MM_EQUIV_label(mismatch):
    if mismatch <= 1: 
        return "<= 1"
    elif mismatch <= 3: 
        return "1-3"
    else:
        return ">3"

In [136]:
dataset_reg['REC_MM_EQUIV_TX'] = dataset_reg['REC_MM_EQUIV_TX'].apply(REC_MM_EQUIV_label)

### DON_HTN

In [137]:
#Donor history of hypertension (1= yes, 0= no)

In [138]:
def DON_HTN_label(label):
    if label == 0:
        return "No"
    elif label == 1: 
        return "Yes"
    else:
        return "Unknown"

In [139]:
dataset_reg['DON_HTN'] = dataset_reg['DON_HTN'].apply(DON_HTN_label)

In [140]:
dataset_reg.DON_HTN.value_counts()

No         37752
Unknown    28761
Yes        14491
Name: DON_HTN, dtype: int64

### DON_HIST_CANCER

In [141]:
def DON_HIST_CANCER_label(label):
    if label == 1.0:
        return "No"
    elif (label == 999) or (label == 998): 
        return "Unknown/Other"
    else:
        return "Yes"

In [142]:
dataset_reg['DON_HIST_CANCER'] = dataset_reg['DON_HIST_CANCER'].apply(DON_HIST_CANCER_label)

In [143]:
dataset_reg.DON_HIST_CANCER.value_counts()

No               78751
Yes               1581
Unknown/Other      672
Name: DON_HIST_CANCER, dtype: int64

### DON_RELATIONSHIP_TY

In [144]:
def DON_RELATIONSHIP_TY_label(label):
    if label < 7:
        return "Biological blood related"
    elif label <= 999: 
        return "Non-Biological"
    else:
        return "Unknown"

In [145]:
dataset_reg['DON_RELATIONSHIP_TY'] = dataset_reg['DON_RELATIONSHIP_TY'].apply(DON_RELATIONSHIP_TY_label)

In [146]:
dataset_reg.DON_RELATIONSHIP_TY.value_counts()

Unknown                     52649
Biological blood related    14507
Non-Biological              13848
Name: DON_RELATIONSHIP_TY, dtype: int64

### REC_MED_COND

In [147]:
#Medical Condition

In [148]:
def REC_MED_COND_label(label):
    if label == 1:
        return "Hospitalized in ICU"
    elif label == 2: 
        return "Not Hospitalized in ICU"
    elif label == 3: 
        return "No Hospitalization"
    else:
        return "Unknown"

In [149]:
dataset_reg['REC_MED_COND'] = dataset_reg['REC_MED_COND'].apply(REC_MED_COND_label)

In [150]:
dataset_reg.REC_MED_COND.value_counts()

No Hospitalization         65286
Unknown                    14253
Not Hospitalized in ICU     1033
Hospitalized in ICU          432
Name: REC_MED_COND, dtype: int64

### REC_ORG_RECEIVED_ON

In [151]:
#Kidney(s) received on: Kidney(s) received on ice, pump

In [152]:
def REC_ORG_RECEIVED_ON_label(label):
    if label == 1:
        return "ice"
    elif label == 2: 
        return "pump"
    else:
        return "N/A"

In [153]:
dataset_reg['REC_ORG_RECEIVED_ON'] = dataset_reg['REC_ORG_RECEIVED_ON'].apply(REC_ORG_RECEIVED_ON_label)

In [154]:

dataset_reg.REC_ORG_RECEIVED_ON.value_counts()

ice     59885
pump    13881
N/A      7238
Name: REC_ORG_RECEIVED_ON, dtype: int64

In [155]:
#check final dataset for null values
dataset_reg.isnull().sum()

REC_AGE_AT_TX                   0
DON_AGE                         0
CAN_LAST_SRTR_PEAK_PRA          0
DON_RACE                        0
CAN_RACE                        0
CAN_GENDER                      0
DON_GENDER                      0
GL                              0
DAYS                            0
DON_BMI                         0
REC_BMI                         0
REC_PRIMARY_PAY                 0
REC_COLD_ISCH_TM                0
REC_TX_DT                       0
REC_PROD_URINE_GT40_24HRS       0
DON_HIGH_CREAT                  0
DON_HIST_CIGARETTE_GT20_PKYR    0
DON_HIST_COCAINE                0
DON_HIST_OTHER_DRUG             0
DON_TY                          0
REC_MALIG                       0
REC_HOSP_90_DAYS                0
Avg_HMS                         0
AbvMismatch                     0
REC_MM_EQUIV_CUR                0
REC_MM_EQUIV_TX                 0
DON_HTN                         0
DON_HIST_CANCER                 0
DON_RELATIONSHIP_TY             0
REC_MED_COND  

### Creating dummy variables

In [156]:
dataset_reg=pd.get_dummies(dataset_reg) #uses drop_first=False by default
dataset_reg.columns

Index(['GL', 'DAYS', 'REC_AGE_AT_TX_18-34', 'REC_AGE_AT_TX_35 - 49',
       'REC_AGE_AT_TX_> 45', 'DON_AGE_18-34', 'DON_AGE_35 - 49',
       'DON_AGE_> 45', 'CAN_LAST_SRTR_PEAK_PRA_0-19',
       'CAN_LAST_SRTR_PEAK_PRA_19 - 95', 'CAN_LAST_SRTR_PEAK_PRA_> 95',
       'DON_RACE_All Others', 'DON_RACE_Black', 'DON_RACE_White',
       'CAN_RACE_All Others', 'CAN_RACE_Black', 'CAN_RACE_White',
       'CAN_GENDER_b'F'', 'CAN_GENDER_b'M'', 'DON_GENDER_b'F'',
       'DON_GENDER_b'M'', 'DON_BMI_0-30', 'DON_BMI_> 30', 'REC_BMI_0-30',
       'REC_BMI_> 30', 'REC_PRIMARY_PAY_All Others',
       'REC_PRIMARY_PAY_Private insurance', 'REC_PRIMARY_PAY_Public insurance',
       'REC_COLD_ISCH_TM_0-24', 'REC_COLD_ISCH_TM_> 24',
       'REC_TX_DT_2011 and after', 'REC_TX_DT_Before 2010',
       'REC_PROD_URINE_GT40_24HRS_No', 'REC_PROD_URINE_GT40_24HRS_Unknown',
       'REC_PROD_URINE_GT40_24HRS_Yes', 'DON_HIGH_CREAT_No',
       'DON_HIGH_CREAT_Unknown', 'DON_HIGH_CREAT_Yes',
       'DON_HIST_CIGARETTE_G

In [157]:
# Address Multi-Collinearity
# drop unnecessary (baseline) fields, this would be the references
dataset_reg = dataset_reg.drop(['REC_AGE_AT_TX_18-34', 'DON_AGE_18-34', 'CAN_LAST_SRTR_PEAK_PRA_0-19', 
                               'DON_RACE_White', 'CAN_RACE_White', "CAN_GENDER_b'F'", "DON_GENDER_b'F'",
                              'DON_BMI_0-30', 'REC_BMI_0-30', 'REC_PRIMARY_PAY_Private insurance', 
                              'REC_COLD_ISCH_TM_0-24', 'REC_TX_DT_Before 2010', "REC_PROD_URINE_GT40_24HRS_No",
                              "DON_HIGH_CREAT_No", "DON_HIST_CIGARETTE_GT20_PKYR_No", 
                               "DON_HIST_COCAINE_No", "DON_HIST_OTHER_DRUG_No", "DON_TY_b'C'",
                              "REC_MALIG_No", "REC_HOSP_90_DAYS_not hosptalized", 'Avg_HMS_>1.8', 
                               'AbvMismatch_<= 5', 'REC_MM_EQUIV_CUR_<= 2', 'REC_MM_EQUIV_TX_<= 1',
                               'DON_HTN_No', 'DON_HIST_CANCER_No', 'DON_RELATIONSHIP_TY_Biological blood related',
                               'REC_MED_COND_Hospitalized in ICU', 'REC_ORG_RECEIVED_ON_ice' ], axis=1)

In [158]:
dataset_reg.columns

Index(['GL', 'DAYS', 'REC_AGE_AT_TX_35 - 49', 'REC_AGE_AT_TX_> 45',
       'DON_AGE_35 - 49', 'DON_AGE_> 45', 'CAN_LAST_SRTR_PEAK_PRA_19 - 95',
       'CAN_LAST_SRTR_PEAK_PRA_> 95', 'DON_RACE_All Others', 'DON_RACE_Black',
       'CAN_RACE_All Others', 'CAN_RACE_Black', 'CAN_GENDER_b'M'',
       'DON_GENDER_b'M'', 'DON_BMI_> 30', 'REC_BMI_> 30',
       'REC_PRIMARY_PAY_All Others', 'REC_PRIMARY_PAY_Public insurance',
       'REC_COLD_ISCH_TM_> 24', 'REC_TX_DT_2011 and after',
       'REC_PROD_URINE_GT40_24HRS_Unknown', 'REC_PROD_URINE_GT40_24HRS_Yes',
       'DON_HIGH_CREAT_Unknown', 'DON_HIGH_CREAT_Yes',
       'DON_HIST_CIGARETTE_GT20_PKYR_Unknown',
       'DON_HIST_CIGARETTE_GT20_PKYR_Yes', 'DON_HIST_COCAINE_Unknown',
       'DON_HIST_COCAINE_Yes', 'DON_HIST_OTHER_DRUG_Unknown',
       'DON_HIST_OTHER_DRUG_Yes', 'DON_TY_b'L'', 'REC_MALIG_Unknown',
       'REC_MALIG_Yes', 'REC_HOSP_90_DAYS_Unknown',
       'REC_HOSP_90_DAYS_hosptalized', 'Avg_HMS_<= 1.8', 'AbvMismatch_5-10',
       '

In [159]:
dataset_reg.head()

,GL,DAYS,REC_AGE_AT_TX_35 - 49,REC_AGE_AT_TX_> 45,DON_AGE_35 - 49,DON_AGE_> 45,CAN_LAST_SRTR_PEAK_PRA_19 - 95,CAN_LAST_SRTR_PEAK_PRA_> 95,DON_RACE_All Others,DON_RACE_Black,...,DON_HTN_Yes,DON_HIST_CANCER_Unknown/Other,DON_HIST_CANCER_Yes,DON_RELATIONSHIP_TY_Non-Biological,DON_RELATIONSHIP_TY_Unknown,REC_MED_COND_No Hospitalization,REC_MED_COND_Not Hospitalized in ICU,REC_MED_COND_Unknown,REC_ORG_RECEIVED_ON_N/A,REC_ORG_RECEIVED_ON_pump
0,0.0,1969,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1.0,1831,0,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2,0.0,1593,1,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,1.0,9,0,1,0,0,1,0,0,1,...,0,0,0,0,1,1,0,0,0,0
4,1.0,836,0,1,0,1,0,0,0,0,...,1,0,0,0,1,1,0,0,0,1


### Feature Selection: Which Variable is Most Predictive?

In [ ]:
https://scikit-survival.readthedocs.io/en/stable/user_guide/00-introduction.html
https://mathweb.ucsd.edu/~rxu/math284/model_sel.pdf
https://scikit-survival.readthedocs.io/en/stable/user_guide/00-introduction.html

In [ ]:
The model below consideres all available variables for prediction.
We briefly describe the stepwise (back-n-forth) procedure to get significant features:
(1) Fit a univariate model for each covariate, and identify the predictors significant at some level using p value 
 and the rank concordance-index.
(2) Fit a multivariate model with all significant univariate predictors, and use backward selection to eliminate
non- significant variables at some level using p-value.

In [77]:
X = dataset_reg

In [78]:
X.head()

,GL,DAYS,REC_AGE_AT_TX_35 - 49,REC_AGE_AT_TX_> 45,DON_AGE_35 - 49,DON_AGE_> 45,CAN_LAST_SRTR_PEAK_PRA_19 - 95,CAN_LAST_SRTR_PEAK_PRA_> 95,DON_RACE_All Others,DON_RACE_Black,...,DON_HTN_Yes,DON_HIST_CANCER_Unknown/Other,DON_HIST_CANCER_Yes,DON_RELATIONSHIP_TY_Non-Biological,DON_RELATIONSHIP_TY_Unknown,REC_MED_COND_No Hospitalization,REC_MED_COND_Not Hospitalized in ICU,REC_MED_COND_Unknown,REC_ORG_RECEIVED_ON_N/A,REC_ORG_RECEIVED_ON_pump
0,0.0,1969,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1.0,1831,0,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2,0.0,1593,1,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,1.0,9,0,1,0,0,1,0,0,1,...,0,0,0,0,1,1,0,0,0,0
4,1.0,836,0,1,0,1,0,0,0,0,...,1,0,0,0,1,1,0,0,0,1


In [79]:
# Fit a univariate model for each covariate, and identify the predictors significant at some level using p value 
# and the rank concordance-index

def fit_and_score_features(X):
    y=X[["GL","DAYS"]]
    X.drop(["GL", "DAYS"], axis=1, inplace=True)
    n_features = X.shape[1]#num of features
    scores = np.empty(n_features)
    AIC = np.empty(n_features)
    m = CoxPHFitter()
    
    for j in range(n_features):
        Xj = X.iloc[:, j:j+1]#extract each column 1 at a time and then merge it with GL and days below
        Xj=pd.merge(Xj, y,  how='right', left_index=True, right_index=True)
        m.fit(Xj, duration_col="DAYS", event_col="GL", show_progress=True) 
        scores[j] = m.concordance_index_
        m.print_summary(3)
    return scores
scores = fit_and_score_features(X)

Iteration 1: norm_delta = 0.15334, step_size = 0.9000, log_lik = -265678.27977, newton_decrement = 305.54304, seconds_since_start = 0.4
Iteration 2: norm_delta = 0.02980, step_size = 0.9000, log_lik = -265363.84233, newton_decrement = 9.83506, seconds_since_start = 0.8
Iteration 3: norm_delta = 0.00357, step_size = 0.9000, log_lik = -265354.01394, newton_decrement = 0.13598, seconds_since_start = 1.2
Iteration 4: norm_delta = 0.00001, step_size = 1.0000, log_lik = -265353.87774, newton_decrement = 0.00000, seconds_since_start = 1.6
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -265353.87774, newton_decrement = 0.00000, seconds_since_start = 2.0
Convergence success after 5 iterations.


<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -265353.878
         time fit was run = 2022-04-19 01:52:09 UTC

---
                        coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                     
REC_AGE_AT_TX_35 - 49 -0.384      0.681      0.016           -0.415           -0.354                0.660                0.702

                            z       p   -log2(p)
covariate                                       
REC_AGE_AT_TX_35 - 49 -24.569 <0.0005    440.374
---
Concordance = 0.534
Partial AIC = 530709.755
log-likelihood ratio test = 648.804 on 1 df
-log2(p) of ll-ratio test = 473.012

Iteration 1: norm_delta = 0.19252, step_size = 0.9000, log_lik = -265678.27977, newton_decrement = 467.02937, seconds_since_start = 0.4
Iteration 2: norm_delta = 0.02552, step_size = 0.9000, log_lik = -265209.70569, newton_decrement = 7.68949, seconds_since_start = 0.8
Iteration 3: norm_delta = 0.00273, step_size = 0.9000, log_lik = -265202.06684, newton_decrement = 0.08707, seconds_since_start = 1.2
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -265201.97972, newton_decrement = 0.00000, seconds_since_start = 1.6
Convergence success after 4 iterations.


<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -265201.980
         time fit was run = 2022-04-19 01:52:12 UTC

---
                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                  
REC_AGE_AT_TX_> 45  0.402      1.495      0.013            0.376            0.428                1.456                1.534

                        z       p   -log2(p)
covariate                                   
REC_AGE_AT_TX_> 45 30.358 <0.0005    670.043
---
Concordance = 0.547
Partial AIC = 530405.959
log-likelihood ratio test = 952.600 on 1 df
-log2(p) of ll-ratio test = 692.431

Iteration 1: norm_delta = 0.03372, step_size = 0.9000, log_lik = -265678.27977, newton_decrement = 14.30160, seconds_since_start = 0.4
Iteration 2: norm_delta = 0.00376, step_size = 0.9000, log_lik = -265664.04262, newton_decrement = 0.17417, seconds_since_start = 0.8
Iteration 3: norm_delta = 0.00038, step_size = 0.9000, log_lik = -265663.87008, newton_decrement = 0.00178, seconds_since_start = 1.2
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -265663.86829, newton_decrement = 0.00000, seconds_since_start = 1.6
Convergence success after 4 iterations.


...

<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -265663.310
         time fit was run = 2022-04-19 01:53:44 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
REC_MED_COND_Unknown  0.295      1.343      0.052            0.193            0.396                1.213                1.487

                         z       p   -log2(p)
covariate                                    
REC_MED_COND_Unknown 5.668 <0.0005     26.044
---
Concordance = 0.503
Partial AIC = 531328.619
log-likelihood ratio test = 29.940 on 1 df
-log2(p) of ll-ratio test = 24.420

Iteration 1: norm_delta = 0.09857, step_size = 0.9000, log_lik = -265678.27977, newton_decrement = 144.75999, seconds_since_start = 0.4
Iteration 2: norm_delta = 0.02574, step_size = 0.9000, log_lik = -265525.88209, newton_decrement = 7.67465, seconds_since_start = 0.8
Iteration 3: norm_delta = 0.00358, step_size = 0.9000, log_lik = -265518.14576, newton_decrement = 0.13870, seconds_since_start = 1.2
Iteration 4: norm_delta = 0.00002, step_size = 1.0000, log_lik = -265518.00658, newton_decrement = 0.00000, seconds_since_start = 1.6
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -265518.00657, newton_decrement = 0.00000, seconds_since_start = 2.1
Convergence success after 5 iterations.


<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -265518.007
         time fit was run = 2022-04-19 01:53:46 UTC

---
                          coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                       
REC_ORG_RECEIVED_ON_N/A -0.405      0.667      0.024           -0.452           -0.358                0.637                0.699

                              z       p   -log2(p)
covariate                                         
REC_ORG_RECEIVED_ON_N/A -16.900 <0.0005    210.430
---
Concordance = 0.518
Partial AIC = 531038.013
log-likelihood ratio test = 320.546 on 1 df
-log2(p) of ll-ratio test = 235.718

Iteration 1: norm_delta = 0.15303, step_size = 0.9000, log_lik = -265678.27977, newton_decrement = 250.81666, seconds_since_start = 0.4
Iteration 2: norm_delta = 0.00261, step_size = 0.9000, log_lik = -265448.14477, newton_decrement = 0.09330, seconds_since_start = 0.8
Iteration 3: norm_delta = 0.00027, step_size = 0.9000, log_lik = -265448.05230, newton_decrement = 0.00097, seconds_since_start = 1.2
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -265448.05133, newton_decrement = 0.00000, seconds_since_start = 1.6
Convergence success after 4 iterations.


<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -265448.051
         time fit was run = 2022-04-19 01:53:49 UTC

---
                           coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                        
REC_ORG_RECEIVED_ON_pump  0.359      1.431      0.016            0.327            0.390                1.387                1.477

                              z       p   -log2(p)
covariate                                         
REC_ORG_RECEIVED_ON_pump 22.278 <0.0005    362.818
---
Concordance = 0.526
Partial AIC = 530898.103
log-likelihood ratio test = 460.457 on 1 df
-log2(p) of ll-ratio test = 336.902

In [80]:
#rank of features using concordance index
pd.Series(scores, index=X.columns).sort_values(ascending=False)

DON_TY_b'L'                             0.564411
DON_HIGH_CREAT_Unknown                  0.564411
DON_RELATIONSHIP_TY_Unknown             0.564403
DON_HTN_Unknown                         0.563646
DON_HIST_OTHER_DRUG_Unknown             0.563362
DON_HIST_COCAINE_Unknown                0.562938
DON_HIST_CIGARETTE_GT20_PKYR_Unknown    0.562333
REC_PRIMARY_PAY_All Others              0.556054
DON_HTN_Yes                             0.548219
REC_AGE_AT_TX_> 45                      0.546652
DON_AGE_> 45                            0.541240
REC_AGE_AT_TX_35 - 49                   0.533904
DON_RELATIONSHIP_TY_Non-Biological      0.529984
DON_HIST_CIGARETTE_GT20_PKYR_Yes        0.529609
REC_MM_EQUIV_TX_>3                      0.528885
REC_ORG_RECEIVED_ON_pump                0.526497
REC_PROD_URINE_GT40_24HRS_Yes           0.526089
REC_MM_EQUIV_CUR_>4                     0.523611
REC_BMI_> 30                            0.520128
REC_MM_EQUIV_TX_1-3                     0.519012
DON_BMI_> 30        

In [ ]:
#From the univariate models fixed, a criteria using combination of c-index and p value helps us to decide which
#variables are significant to be further used in model feature selection
#variables not significant and not having a great concordancev index wd be would be dropped 

Using c-index, DON_TY is the best significant variable. Other top significant variables to our model are 
DON_HIGH_CREAT, DON_RELATIONSHIP, DON_HTN, DON drug HIST, REC age at TX 

whereas DON_RACE, CAN_RACE, REC_TX_DT_2011 have almost no predictive power on 
their own.

The bottom features in the c-index rank do not perfom well, Taking a look at their p-value confirms they are 
not significant in their univariate modsels as well.
To that effect we see, REC_MALIG_Yes, DON_RACE_All Others, CAN_RACE_All Others, AbvMismatch_10-20, REC_TX_DT_2011 
and after so we start by dropping those 

so we dropped DON_RACE, CAN_RACE, REC_TX_DT_2011, REC_MALIG, AbvMismatch etc

Next, we want to build a parsimonious model by excluding irrelevant features. 

### FIT MODEL WITH SIGNIFICANT FEATURES

In [81]:
dataset_reg.columns

Index(['REC_AGE_AT_TX_35 - 49', 'REC_AGE_AT_TX_> 45', 'DON_AGE_35 - 49',
       'DON_AGE_> 45', 'CAN_LAST_SRTR_PEAK_PRA_19 - 95',
       'CAN_LAST_SRTR_PEAK_PRA_> 95', 'DON_RACE_All Others', 'DON_RACE_Black',
       'CAN_RACE_All Others', 'CAN_RACE_Black', 'CAN_GENDER_b'M'',
       'DON_GENDER_b'M'', 'DON_BMI_> 30', 'REC_BMI_> 30',
       'REC_PRIMARY_PAY_All Others', 'REC_PRIMARY_PAY_Public insurance',
       'REC_COLD_ISCH_TM_> 24', 'REC_TX_DT_2011 and after',
       'REC_PROD_URINE_GT40_24HRS_Unknown', 'REC_PROD_URINE_GT40_24HRS_Yes',
       'DON_HIGH_CREAT_Unknown', 'DON_HIGH_CREAT_Yes',
       'DON_HIST_CIGARETTE_GT20_PKYR_Unknown',
       'DON_HIST_CIGARETTE_GT20_PKYR_Yes', 'DON_HIST_COCAINE_Unknown',
       'DON_HIST_COCAINE_Yes', 'DON_HIST_OTHER_DRUG_Unknown',
       'DON_HIST_OTHER_DRUG_Yes', 'DON_TY_b'L'', 'REC_MALIG_Unknown',
       'REC_MALIG_Yes', 'REC_HOSP_90_DAYS_Unknown',
       'REC_HOSP_90_DAYS_hosptalized', 'Avg_HMS_<= 1.8', 'AbvMismatch_5-10',
       'AbvMismatch_>1

In [160]:
#selecting significant features
dataset_sig = dataset_reg[[
       'GL', 'DAYS', 'REC_AGE_AT_TX_35 - 49', 'REC_AGE_AT_TX_> 45', 'DON_AGE_35 - 49',
       'DON_AGE_> 45', 'CAN_LAST_SRTR_PEAK_PRA_19 - 95',
       'CAN_LAST_SRTR_PEAK_PRA_> 95', "CAN_GENDER_b'M'",
       "DON_GENDER_b'M'", 'DON_BMI_> 30', 'REC_BMI_> 30',
       'REC_PRIMARY_PAY_All Others', 'REC_PRIMARY_PAY_Public insurance',
       'REC_COLD_ISCH_TM_> 24',
       'REC_PROD_URINE_GT40_24HRS_Unknown', 'REC_PROD_URINE_GT40_24HRS_Yes',
       'DON_HIGH_CREAT_Unknown', 'DON_HIGH_CREAT_Yes',
       'DON_HIST_CIGARETTE_GT20_PKYR_Unknown',
       'DON_HIST_CIGARETTE_GT20_PKYR_Yes', 'DON_HIST_OTHER_DRUG_Unknown',
       'DON_HIST_OTHER_DRUG_Yes', "DON_TY_b'L'", 'REC_HOSP_90_DAYS_Unknown',
       'REC_HOSP_90_DAYS_hosptalized', 'Avg_HMS_<= 1.8', 'REC_MM_EQUIV_CUR_2-4', 'REC_MM_EQUIV_CUR_>4',
       'REC_MM_EQUIV_TX_1-3', 'REC_MM_EQUIV_TX_>3', 'DON_HTN_Unknown',
       'DON_HTN_Yes', 
       'DON_RELATIONSHIP_TY_Non-Biological', 'DON_RELATIONSHIP_TY_Unknown',
       'REC_ORG_RECEIVED_ON_N/A', 'REC_ORG_RECEIVED_ON_pump']]

In [ ]:
#Fitting Cox model with penalizer 0.1

In [161]:
cph = CoxPHFitter(penalizer=0.1)
cph.fit(dataset_sig, duration_col='DAYS', event_col='GL')

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -263768.43
         time fit was run = 2022-04-19 02:00:39 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                    
REC_AGE_AT_TX_35 - 49                 -0.18       0.84       0.02            -0.21            -0.15                 0.81                 0.86
REC_AGE_AT_TX_> 45                     0.11       1.12       0.01             0.08             0.14                 1.09                 1.15
DON_AGE_35 - 49                        0.07       1.07       0.01             0.04             0.09                 1.04                 1.10
DON_AGE_> 45                           0.23       1.25       0.01             0.20             0.25                 1.22                 1.29
CAN_LAST_SRTR_PEAK_PRA_19 - 95         0.04       1.04       0.01             0.02             0.06                 1.02                 1.07
CAN_LAST_SRTR_PEAK_PRA_> 95            0.14       1.15       0.03             0.09             0.19                 1.10                 1.21
CAN_GENDER_b'M'                        0.08       1.09       0.01             0.06             0.11                 1.06                 1.11
DON_GENDER_b'M'                       -0.04       0.96       0.01            -0.06            -0.02                 0.94                 0.98
DON_BMI_> 30                           0.05       1.05       0.01             0.02             0.07                 1.02                 1.07
REC_BMI_> 30                           0.10       1.11       0.01             0.08             0.13                 1.09                 1.14
REC_PRIMARY_PAY_All Others             0.24       1.27       0.01             0.22             0.26                 1.24                 1.30
REC_PRIMARY_PAY_Public insurance       0.07       1.07       0.03             0.02             0.12                 1.02                 1.13
REC_COLD_ISCH_TM_> 24                  0.06       1.07       0.01             0.04             0.09                 1.04                 1.09
REC_PROD_URINE_GT40_24HRS_Unknown     -0.09       0.92       0.02            -0.12            -0.06                 0.89                 0.95
REC_PROD_URINE_GT40_24HRS_Yes         -0.14       0.87       0.01            -0.17            -0.11                 0.84                 0.89
DON_HIGH_CREAT_Unknown                -0.06       0.94       0.02            -0.10            -0.02                 0.90                 0.98
DON_HIGH_CREAT_Yes                     0.05       1.05       0.02             0.02             0.08                 1.02                 1.09
DON_HIST_CIGARETTE_GT20_PKYR_Unknown  -0.03       0.97       0.02            -0.07             0.01                 0.94                 1.01
DON_HIST_CIGARETTE_GT20_PKYR_Yes       0.08       1.09       0.02             0.05             0.11                 1.05                 1.12
DON_HIST_OTHER_DRUG_Unknown           -0.05       0.95       0.02            -0.09            -0.01                 0.92                 0.99
DON_HIST_OTHER_DRUG_Yes               -0.02       0.98       0.01            -0.05             0.00                 0.95                 1.00
DON_TY_b'L'                           -0.06       0.94       0.02            -0.10            -0.02                 0.90                 0.98
REC_HOSP_90_DAYS_Unknown               0.01       1.01       0.01            -0.02             0.04                 0.98                 1.04
REC_HOSP_90_DAYS_hosp

In [162]:
cph = CoxPHFitter(penalizer=0.05)
cph.fit(dataset_sig, duration_col='DAYS', event_col='GL')

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
                penalizer = 0.05
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -263622.69
         time fit was run = 2022-04-19 02:00:54 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                    
REC_AGE_AT_TX_35 - 49                 -0.21       0.81       0.02            -0.25            -0.18                 0.78                 0.84
REC_AGE_AT_TX_> 45                     0.10       1.11       0.02             0.07             0.13                 1.07                 1.14
DON_AGE_35 - 49                        0.09       1.10       0.01             0.07             0.12                 1.07                 1.13
DON_AGE_> 45                           0.26       1.30       0.02             0.23             0.29                 1.26                 1.34
CAN_LAST_SRTR_PEAK_PRA_19 - 95         0.05       1.05       0.01             0.03             0.08                 1.03                 1.08
CAN_LAST_SRTR_PEAK_PRA_> 95            0.17       1.19       0.03             0.12             0.23                 1.12                 1.25
CAN_GENDER_b'M'                        0.10       1.10       0.01             0.07             0.12                 1.07                 1.13
DON_GENDER_b'M'                       -0.05       0.96       0.01            -0.07            -0.02                 0.93                 0.98
DON_BMI_> 30                           0.05       1.05       0.01             0.02             0.07                 1.02                 1.07
REC_BMI_> 30                           0.12       1.13       0.01             0.09             0.14                 1.10                 1.15
REC_PRIMARY_PAY_All Others             0.28       1.32       0.01             0.25             0.30                 1.29                 1.36
REC_PRIMARY_PAY_Public insurance       0.11       1.11       0.03             0.05             0.17                 1.05                 1.18
REC_COLD_ISCH_TM_> 24                  0.07       1.07       0.02             0.04             0.10                 1.04                 1.10
REC_PROD_URINE_GT40_24HRS_Unknown     -0.15       0.86       0.02            -0.19            -0.11                 0.83                 0.90
REC_PROD_URINE_GT40_24HRS_Yes         -0.20       0.82       0.02            -0.23            -0.16                 0.79                 0.85
DON_HIGH_CREAT_Unknown                -0.07       0.94       0.03            -0.12            -0.01                 0.88                 0.99
DON_HIGH_CREAT_Yes                     0.05       1.05       0.02             0.02             0.09                 1.02                 1.09
DON_HIST_CIGARETTE_GT20_PKYR_Unknown  -0.01       0.99       0.03            -0.06             0.05                 0.94                 1.05
DON_HIST_CIGARETTE_GT20_PKYR_Yes       0.08       1.08       0.02             0.04             0.11                 1.04                 1.11
DON_HIST_OTHER_DRUG_Unknown           -0.05       0.96       0.03            -0.10             0.01                 0.90                 1.01
DON_HIST_OTHER_DRUG_Yes               -0.02       0.98       0.01            -0.05             0.00                 0.95                 1.00
DON_TY_b'L'                           -0.07       0.94       0.03            -0.12            -0.01                 0.88                 0.99
REC_HOSP_90_DAYS_Unknown               0.02       1.02       0.02            -0.01             0.05                 0.99                 1.05
REC_HOSP_90_DAYS_hos

In [ ]:
#using lasso regression 

In [314]:
#cph = CoxPHFitter(penalizer=0.1, l1_ratio=1.0)
#cph.fit(dataset_reg, duration_col='DAYS', event_col='GL')

#cph.print_summary()

In [163]:
cph = CoxPHFitter(penalizer=0.005, l1_ratio=0.5)
cph.fit(dataset_reg, duration_col='DAYS', event_col='GL')

cph.print_summary(4)

<lifelines.CoxPHFitter: fitted with 81004 total observations, 56025 right-censored observations>
             duration col = 'DAYS'
                event col = 'GL'
                penalizer = 0.005
                 l1 ratio = 0.5
      baseline estimation = breslow
   number of observations = 81004
number of events observed = 24979
   partial log-likelihood = -263703.4499
         time fit was run = 2022-04-19 02:14:01 UTC

---
                                        coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                     
REC_AGE_AT_TX_35 - 49                -0.2492     0.7795     0.0218          -0.2919          -0.2064               0.7468               0.8135
REC_AGE_AT_TX_> 45                    0.0698     1.0723     0.0192           0.0321           0.1075               1.0326               1.1135
DON_AGE_35 - 49                       0.1130     1.1197     0.0169           0.0799           0.1462               1.0832               1.1574
DON_AGE_> 45                          0.3015     1.3519     0.0178           0.2667           0.3364               1.3056               1.3999
CAN_LAST_SRTR_PEAK_PRA_19 - 95        0.0345     1.0351     0.0145           0.0060           0.0630               1.0061               1.0651
CAN_LAST_SRTR_PEAK_PRA_> 95           0.1432     1.1539     0.0297           0.0850           0.2013               1.0887               1.2230
DON_RACE_All Others                   0.0000     1.0000     0.0003          -0.0007           0.0007               0.9993               1.0007
DON_RACE_Black                        0.0087     1.0087     0.0182          -0.0270           0.0443               0.9734               1.0453
CAN_RACE_All Others                  -0.0000     1.0000     0.0002          -0.0003           0.0003               0.9997               1.0003
CAN_RACE_Black                        0.0000     1.0000     0.0001          -0.0003           0.0003               0.9997               1.0003
CAN_GENDER_b'M'                       0.0884     1.0925     0.0134           0.0622           0.1147               1.0642               1.1215
DON_GENDER_b'M'                      -0.0333     0.9672     0.0130          -0.0589          -0.0078               0.9428               0.9922
DON_BMI_> 30                          0.0333     1.0338     0.0144           0.0050           0.0615               1.0050               1.0635
REC_BMI_> 30                          0.1187     1.1261     0.0132           0.0929           0.1445               1.0974               1.1555
REC_PRIMARY_PAY_All Others            0.3125     1.3669     0.0146           0.2839           0.3412               1.3283               1.4066
REC_PRIMARY_PAY_Public insurance      0.1016     1.1070     0.0330           0.0369           0.1663               1.0376               1.1810
REC_COLD_ISCH_TM_> 24                 0.0594     1.0612     0.0161           0.0277           0.0910               1.0281               1.0953
REC_TX_DT_2011 and after              0.0000     1.0000     0.0001          -0.0003           0.0003               0.9997               1.0003
REC_PROD_URINE_GT40_24HRS_Unknown    -0.2062     0.8137     0.0253          -0.2558          -0.1566               0.7743               0.8551
REC_PROD_URINE_GT40_24HRS_Yes        -0.2659     0.7665     0.0233          -0.3117          -0.2202               0.7322               0.8024
DON_HIGH_CREAT_Unknown               -0.0975     0.9071     0.1204          -0.3336           0.1385               0.7164               1.1486
DON_HIGH_CREAT_Yes                    0.0377     1.0384     0.0201          -0.0017           0.0771               0.9983               1.0802
DON_HIST_CIGARETTE_GT20_PKYR_Unknown -0.0000     1.0000     0.0001          -0.0003           0.0002               0.9997              

In [ ]:
#first one
#set to zero
CAN_RACE_All Others
CAN_RACE_Black
DON_RACE_All Others
REC_TX_DT_2011 and after
DON_HIST_CIGARETTE_GT20_PKYR_Unknown
DON_HIST_COCAINE_Yes
DON_HIST_COCAINE_Unknown
REC_HOSP_90_DAYS_Unknown
AbvMismatch
REC_MM_EQUIV_CUR_2-4
REC_MM_EQUIV_TX_1-3
REC_ORG_RECEIVED_ON_N/A
DON_HTN_Unknown
DON_HIST_CANCER_Yes
REC_MED_COND_Not Hospitalized in ICU
REC_MED_COND_Unknown
REC_ORG_RECEIVED_ON_N/A

In [ ]:
#features set to zero
CAN_RACE_All Others
CAN_RACE_Black
DON_RACE_All Others
REC_TX_DT_2011 and after
DON_HIST_CIGARETTE_GT20_PKYR_Unknown
DON_HIST_COCAINE_Yes
DON_HIST_COCAINE_Unknown
REC_HOSP_90_DAYS_Unknown
AbvMismatch
REC_MM_EQUIV_CUR_2-4
REC_MM_EQUIV_TX_1-3
REC_ORG_RECEIVED_ON_N/A
DON_HTN_Unknown
DON_HIST_CANCER_Yes
REC_MED_COND_Not Hospitalized in ICU
REC_MED_COND_Unknown
REC_ORG_RECEIVED_ON_N/A


In [171]:
#This type of regularization (L1) can lead to zero coefficients i.e. some of the features are completely neglected for the evaluation of output. So Lasso regression not only helps in reducing over-fitting but it can help us in feature selection.

In [167]:
#pip install git+https://github.com/sebp/scikit-survival.git

In [168]:
#pip install scikit-survival

In [166]:
#from sksurv.linear_model import CoxPHSurvivalAnalysis

In [150]:
#pip uninstall typing

In [ ]:
#pip uninstall typing
#pip install -U scikit-survival

In [151]:
#pip install scikit-survival

In [ ]:
GRADIENT BOOSTING FOR SURVIVAL ANALYSIS

In [ ]:
https://escholarship.org/uc/item/6cd436dj
https://towardsdatascience.com/xgbse-improving-xgboost-for-survival-analysis-393d47f1384a
https://towardsdatascience.com/survival-modeling-accelerated-failure-time-xgboost-971aaa1ba794
https://digitalcommons.usf.edu/cgi/viewcontent.cgi?article=9259&context=etd